In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

# Define the path to save the fine-tuned model
load_model = os.path.join("..", "..", "models", "fine_tuned_model")  # Points to InsureAI/models/fine_tuned_model

# Load the fine-tuned model and tokenizer
model = AutoModelForCausalLM.from_pretrained(load_model)
tokenizer = AutoTokenizer.from_pretrained(load_model)

# Generate a response
prompt = "Describe the insurance product Death A offered by InsureAI."
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=512)

# Decode and print the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)